In [2]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import os

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [83]:
# テスト用
url = "https://nicoly.jp/clinic/1525"

# user agent指定
headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:47.0) Gecko/20100101 Firefox/47.0",
        }

res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.content, "html.parser")

table = soup.find_all("table")[0]
rows = table.find_all("tr")

In [87]:
# 本番
baseUrl = "https://nicoly.jp/clinic/"
# user agent指定
headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:47.0) Gecko/20100101 Firefox/47.0",
        }

df_all = pd.DataFrame()

for i in tqdm(range(1520, 1530)):
    url = baseUrl+str(i)
    
    if i % 100 == 0:
        time.sleep(10)
    res = requests.get(url, headers=headers)
    if res.status_code == 200:
        soup = BeautifulSoup(res.content, "html.parser")
        table = soup.find_all("table")[0]
        rows = table.find_all("tr")

        dict = {}
        for row in rows:
            key = row.th.get_text()
            tmp = re.sub(r"※.*", "", row.td.get_text())
            tmp = re.sub(r"\s+", " ", tmp)
            tmp = re.sub(r"\s+$", "", tmp)
            dict[key] = tmp

            if row.td.a:
                dict[key] = row.td.a.get("href")

        df = pd.io.json.json_normalize(dict)

        try:
            df_all = pd.concat([df_all, df])
        except NameError:
            df_all = pd.io.json.json_normalize(dict)

100%|██████████| 10/10 [00:07<00:00,  1.38it/s]


# データの集計

In [6]:
target_dir = "/Users/YukiOta/Desktop/nicoly"
filelist = os.listdir("/Users/YukiOta/Desktop/nicoly")

In [8]:
df_all = pd.DataFrame()

In [24]:
# 重複した行を削除
df_comp = df_all.drop_duplicates()

In [11]:
for file_ in filelist:
    tmp = pd.read_csv(os.path.join(target_dir, file_))
    df_all = pd.concat([df_all, tmp])

In [28]:
len(df_comp)

3179

In [26]:
# df_all.to_csv("./out/nicoly_test.csv")
df_comp.to_csv("./data_nicoly.csv")